<a href="https://colab.research.google.com/github/elephanti/NLPProject2024/blob/main/Train_and_Test_with_augmented_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/elephanti/NLPProject2024.git

Cloning into 'NLPProject2024'...
remote: Enumerating objects: 1073, done.
remote: Counting objects: 100% (1073/1073), done.
remote: Compressing objects: 100% (882/882), done.
remote: Total 1073 (delta 312), reused 873 (delta 184), pack-reused 0 (from 0)
Receiving objects: 100% (1073/1073), 32.84 MiB | 16.00 MiB/s, done.
Resolving deltas: 100% (312/312), done.


In [2]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip -d NLPProject2024/classifiers/embeddings

--2024-08-14 17:26:53--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-08-14 17:26:54--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-08-14 17:26:54--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [1]:
!pip install tensorflow==2.15.1
!pip install ktrain


In [2]:
!pip install transformers==4.37.2

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


def load_data(full_train_path, generated_train_path=None, test_path=None, val_path=None):
    print(f"\tFull train path: {full_train_path}")
    print(f"\tGenerated train path: {generated_train_path}")
    print(f"\tTest path: {test_path}")
    train_df = pd.read_csv(full_train_path)

    if generated_train_path:
        train_df = pd.concat([train_df, pd.read_csv(generated_train_path)], ignore_index=True)

    train_text = train_df['text'].tolist()
    train_label = train_df['label']

    le = LabelEncoder()
    y_train = le.fit_transform(train_label)

    test_df = pd.read_csv(test_path)
    test_text = test_df['text'].tolist()
    test_label = test_df['label']
    y_test = le.transform(test_label)

    if val_path:
      val_df = pd.read_csv(val_path)
      val_text = val_df['text'].tolist()
      val_label = val_df['label']
      y_val = le.transform(val_label)
      return train_text, y_train, test_text, y_test, le, val_text, y_val

    return train_text, y_train, test_text, y_test, le

In [7]:
import pandas as pd
import glob
import os
import joblib
from NLPProject2024.classifiers.bert_ktrain import DistilBERT
from NLPProject2024.classifiers.lstm_glove import LSTMGlove
from NLPProject2024.classifiers.svm_glove import SVMGlove
from NLPProject2024.classifiers.svm_tfidf import SVMTFIDF
import time

class ModelTrainer:
    def __init__(self, classifier_name, glove_file, num_labels):
        self.classifier = None
        self.classifier_name = classifier_name
        self.glove_file = glove_file
        self.num_labels = num_labels

    def train_classifier(self, X_train, y_train, model_path, X_test=None, y_test=None, X_val=None, y_val=None):
        print(f"Training {self.classifier_name}...")
        start_time = time.time()
        if self.classifier_name == 'lstm_glove':
            self.classifier = LSTMGlove
            clf_instance = self.classifier(self.glove_file, self.num_labels)
        elif self.classifier_name == 'svm_glove':
            self.classifier = SVMGlove
            clf_instance = self.classifier(self.glove_file)
        elif self.classifier_name == 'bert_ktrain':
            self.classifier = DistilBERT
            clf_instance = self.classifier(maxlen=50, batch_size=10, learning_rate=5e-5, epochs=20,
                                           early_stopping=4, reduce_on_plateau=2)
        else:
            self.classifier = SVMTFIDF
            clf_instance = self.classifier()

        clf_instance.train(X_train, y_train, X_val, y_val)

        accuracy, precision, recall, f1 = clf_instance.evaluate(X_test, y_test)
        if self.classifier_name in ['lstm_glove', 'bert_ktrain']:
            results = {"classifier": self.classifier_name,
                       "test_accuracy": round(accuracy.item(), 4),
                       "precision": round(precision.item(), 4),
                       "recall": round(recall.item(), 4),
                       "f1_score": round(f1.item(), 4)}
        else:
            results = {"classifier": self.classifier_name,
                       "test_accuracy": round(accuracy, 4),
                       "precision": round(precision, 4),
                       "recall": round(recall, 4),
                       "f1_score": round(f1, 4)}

        clf_instance.save(f'{model_path}/{self.classifier_name}')
        end_time = time.time()
        results["training_time"] = round(end_time - start_time, 2)

        return results


In [5]:
df_result = pd.DataFrame(columns=['dataset', 'model', 'test_accuracy', 'precision', 'recall', 'f1_score'])
glove_file = 'NLPProject2024/classifiers/embeddings/glove.6B.100d.txt'

classifiers = {
    'svm_tfidf': SVMTFIDF,
    'svm_glove': SVMGlove,
    'lstm_glove': LSTMGlove,
    'bert_ktrain': DistilBERT
}

lambadas = ['Lambada', 'Lambada+', 'Lambada+Instruct']
llms = ['Llama3', 'Mistral', 'GPT2']
dataset_names = ['ATIS', 'TREC']

In [14]:
full_df_result = pd.DataFrame(columns=['dataset', 'model', 'test_accuracy', 'precision', 'recall', 'f1_score'])

# Train on full training data
for name in dataset_names:
    X_train, y_train, X_test, y_test, le, X_val, y_val = load_data(f'NLPProject2024/datasets/{name}/{name.lower()}.train.csv',
                                                                   test_path=f'NLPProject2024/datasets/{name}/{name.lower()}.test.csv',
                                                                   val_path=f'NLPProject2024/datasets/{name}/{name.lower()}.valid.csv')
    model_path = f'final_results/models/{name.lower()}_train'
    os.makedirs(model_path, exist_ok=True)

    joblib.dump(le, f'{model_path}/label_encoder.pkl')

    for classifier in classifiers:
        print(f"Training {name} with {classifier}...")
        trainer = ModelTrainer(classifier, glove_file, len(le.classes_))
        train_results = trainer.train_classifier(X_train, y_train, model_path, X_test=X_test, y_test=y_test, X_val=X_val, y_val=y_val)

        train_results["dataset"] = f'{name.lower()}_full'
        full_df_result = pd.concat([full_df_result, pd.DataFrame([train_results])], ignore_index=True)
        print(f"Finished training {name} with {classifier}")
        print(train_results)
        print("-" * 50)

	Full train path: NLPProject2024/datasets/ATIS/atis.train.csv
	Generated train path: None
	Test path: NLPProject2024/datasets/ATIS/atis.test.csv
Training ATIS with svm_tfidf...
Training svm_tfidf...

Validation Results:
Accuracy: 0.9743
Precision: 0.9704
Recall: 0.9743
F1 Score: 0.9711


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.9588
Precision: 0.9516
Recall: 0.9588
F1 Score: 0.9532
Model saved.
Finished training ATIS with svm_tfidf
{'classifier': 'svm_tfidf', 'test_accuracy': 0.9588, 'precision': 0.9516, 'recall': 0.9588, 'f1_score': 0.9532, 'training_time': 6.87, 'dataset': 'atis_full'}
--------------------------------------------------
Training ATIS with svm_glove...
Training svm_glove...

Validation Results:
Accuracy: 0.9383
Precision: 0.9332
Recall: 0.9383
F1 Score: 0.9349


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.9280
Precision: 0.9121
Recall: 0.9280
F1 Score: 0.9170
Model saved.
Finished training ATIS with svm_glove
{'classifier': 'svm_glove', 'test_accuracy': 0.928, 'precision': 0.9121, 'recall': 0.928, 'f1_score': 0.917, 'training_time': 11.51, 'dataset': 'atis_full'}
--------------------------------------------------
Training ATIS with lstm_glove...
Training lstm_glove...
Epoch 1/20, Loss: 0.9069, Accuracy: 0.7774
Accuracy: 0.9280
Precision: 0.9068
Recall: 0.9280
F1 Score: 0.9128
Validation - Accuracy: 0.9280, Precision: 0.9068, Recall: 0.9280, F1 Score: 0.9128


...

Accuracy: 0.9517
Precision: 0.9541
Recall: 0.9517
F1 Score: 0.9499
Validation - Accuracy: 0.9517, Precision: 0.9541, Recall: 0.9517, F1 Score: 0.9499
Epoch 20/20, Loss: 0.0944, Accuracy: 0.9703


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.9536
Precision: 0.9546
Recall: 0.9536
F1 Score: 0.9527
Validation - Accuracy: 0.9536, Precision: 0.9546, Recall: 0.9536, F1 Score: 0.9527
Accuracy: 0.9630
Precision: 0.9649
Recall: 0.9630
F1 Score: 0.9631
Model and embeddings saved.
Finished training TREC with lstm_glove
{'classifier': 'lstm_glove', 'test_accuracy': 0.963, 'precision': 0.9649, 'recall': 0.963, 'f1_score': 0.9631, 'training_time': 109.84, 'dataset': 'trec_full'}
--------------------------------------------------
Training TREC with bert_ktrain...
Training bert_ktrain...
preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 17
	99percentile : 22


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 10
	95percentile : 17
	99percentile : 21




begin training using triangular learning rate policy with max lr of 5e-05...
Epoch 1/20
1277/1277 [==============================] - 74s 49ms/step - loss: 0.8463 - accuracy: 0.8111 - val_loss: 0.2134 - val_accuracy: 0.9524
Epoch 2/20
1277/1277 [==============================] - 61s 47ms/step - loss: 0.1260 - accuracy: 0.9735 - val_loss: 0.1160 - val_accuracy: 0.9743
Epoch 3/20
1277/1277 [==============================] - 62s 47ms/step - loss: 0.0697 - accuracy: 0.9858 - val_loss: 0.0904 - val_accuracy: 0.9824
Epoch 4/20
1277/1277 [==============================] - 62s 48ms/step - loss: 0.0378 - accuracy: 0.9924 - val_loss: 0.0901 - val_accuracy: 0.9806
Epoch 5/20
1277/1277 [==============================] - 62s 48ms/step - loss: 0.0300 - accuracy: 0.9932 - val_loss: 0.0735 - val_accuracy: 0.9881
Epoch 6/20
1277/1277 [==============================] - 61s 47ms/step - loss: 0.0280 - accuracy: 0.9937 - val_loss: 0.0748 - val_accuracy: 0.9862
Epoch 7/20
1276/1277 [=======================

In [15]:
full_df_result.head(10)

,dataset,model,test_accuracy,precision,recall,f1_score,classifier,training_time
0,atis_full,NaN,0.9588,0.9516,0.9588,0.9532,svm_tfidf,6.87
1,atis_full,NaN,0.9280,0.9121,0.9280,0.9170,svm_glove,11.51
2,atis_full,NaN,0.9811,0.9811,0.9811,0.9805,lstm_glove,60.87
3,atis_full,NaN,0.9897,0.9888,0.9897,0.9891,bert_ktrain,262.77
4,trec_full,NaN,0.9599,0.9611,0.9599,0.9598,svm_tfidf,81.70
5,trec_full,NaN,0.8177,0.8224,0.8177,0.8168,svm_glove,40.77
6,trec_full,NaN,0.9630,0.9649,0.9630,0.9631,lstm_glove,109.84
7,trec_full,NaN,0.9831,0.9838,0.9831,0.9831,bert_ktrain,875.90


In [12]:
subsets_df_result = pd.DataFrame(columns=['dataset', 'classifier', 'model', 'subset_size', 'method', 'test_accuracy',
                                          'precision', 'recall', 'f1_score'])

# Train on the subsets
for name in dataset_names:
    for subset_size in [5]:
        subset_path = f'NLPProject2024/datasets/{name}/sampled_subsets/ver1/{name.lower()}_{subset_size}_subset.csv'

        # subset only training
        X_train, y_train, X_test, y_test, le, X_val, y_val = load_data(subset_path,
                                                                       test_path=f'NLPProject2024/datasets/{name}/{name.lower()}.test.csv',
                                                                       val_path=f'NLPProject2024/datasets/{name}/{name.lower()}.valid.csv')
        model_path = f'final_results/models/{name.lower()}_{subset_size}_subset_train'
        os.makedirs(model_path, exist_ok=True)

        joblib.dump(le, f'{model_path}/label_encoder.pkl')

        for classifier in classifiers:
            print(f"Training {name} with {classifier} and subset size {subset_size}...")
            trainer = ModelTrainer(classifier, glove_file, len(le.classes_))
            train_results = trainer.train_classifier(X_train, y_train, model_path, X_test=X_test, y_test=y_test, X_val=X_val, y_val=y_val)

            train_results["dataset"] = f'{name.lower()}_{subset_size}_subset_train'
            train_results["method"] = None
            train_results["model"] = None
            train_results["subset_size"] = subset_size
            subsets_df_result = pd.concat([subsets_df_result, pd.DataFrame([train_results])], ignore_index=True)
            print(f"Finished training {name} with {classifier} and subset size {subset_size}")
            print(train_results)
            print("-" * 50)

	Full train path: NLPProject2024/datasets/ATIS/sampled_subsets/ver1/atis_5_subset.csv
	Generated train path: None
	Test path: NLPProject2024/datasets/ATIS/atis.test.csv
Training ATIS with svm_tfidf and subset size 5...
Training svm_tfidf...

Validation Results:
Accuracy: 0.5798
Precision: 0.8624
Recall: 0.5798
F1 Score: 0.6576
Accuracy: 0.6055
Precision: 0.8235
Recall: 0.6055
F1 Score: 0.6602
Model saved.
Finished training ATIS with svm_tfidf and subset size 5
{'classifier': 'svm_tfidf', 'test_accuracy': 0.6055, 'precision': 0.8235, 'recall': 0.6055, 'f1_score': 0.6602, 'training_time': 0.06, 'dataset': 'atis_5_subset_train', 'method': None, 'model': None, 'subset_size': 5}
--------------------------------------------------
Training ATIS with svm_glove and subset size 5...
Training svm_glove...


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Validation Results:
Accuracy: 0.5832
Precision: 0.8216
Recall: 0.5832
F1 Score: 0.6616
Accuracy: 0.5815
Precision: 0.7998
Recall: 0.5815
F1 Score: 0.6436
Model saved.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Finished training ATIS with svm_glove and subset size 5
{'classifier': 'svm_glove', 'test_accuracy': 0.5815, 'precision': 0.7998, 'recall': 0.5815, 'f1_score': 0.6436, 'training_time': 8.44, 'dataset': 'atis_5_subset_train', 'method': None, 'model': None, 'subset_size': 5}
--------------------------------------------------
Training ATIS with lstm_glove and subset size 5...
Training lstm_glove...
Epoch 1/20, Loss: 3.2355, Accuracy: 0.0244
Accuracy: 0.1149
Precision: 0.6463
Recall: 0.1149
F1 Score: 0.1660
Validation - Accuracy: 0.1149, Precision: 0.6463, Recall: 0.1149, F1 Score: 0.1660
Epoch 2/20, Loss: 2.8502, Accuracy: 0.0976


...

Accuracy: 0.3803
Precision: 0.5676
Recall: 0.3803
F1 Score: 0.3964
Model and embeddings saved.
Finished training TREC with lstm_glove and subset size 5
{'classifier': 'lstm_glove', 'test_accuracy': 0.3803, 'precision': 0.5676, 'recall': 0.3803, 'f1_score': 0.3964, 'training_time': 42.89, 'dataset': 'trec_5_subset_train', 'method': None, 'model': None, 'subset_size': 5}
--------------------------------------------------
Training TREC with bert_ktrain and subset size 5...
Training bert_ktrain...
preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 21


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 10
	95percentile : 17
	99percentile : 21




begin training using triangular learning rate policy with max lr of 5e-05...
Epoch 1/20
25/25 [==============================] - 12s 181ms/step - loss: 3.9439 - accuracy: 0.0000e+00 - val_loss: 3.8931 - val_accuracy: 0.0364
Epoch 2/20
25/25 [==============================] - 3s 103ms/step - loss: 3.7248 - accuracy: 0.1960 - val_loss: 3.6564 - val_accuracy: 0.1599
Epoch 3/20
25/25 [==============================] - 3s 103ms/step - loss: 3.0180 - accuracy: 0.4880 - val_loss: 3.2480 - val_accuracy: 0.2602
Epoch 4/20
25/25 [==============================] - 3s 104ms/step - loss: 2.1063 - accuracy: 0.7560 - val_loss: 2.8270 - val_accuracy: 0.3379
Epoch 5/20
25/25 [==============================] - 3s 103ms/step - loss: 1.3136 - accuracy: 0.9240 - val_loss: 2.4640 - val_accuracy: 0.4144
Epoch 6/20
25/25 [==============================] - 3s 106ms/step - loss: 0.7801 - accuracy: 0.9760 - val_loss: 2.1808 - val_accuracy: 0.4878
Epoch 7/20
25/25 [==============================] - 3s 103ms/ste

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.54%
Precision: 0.59
Recall: 0.54
F1 Score: 0.53
Model saved to final_results/models/trec_5_subset_train/bert_ktrain
Finished training TREC with bert_ktrain and subset size 5
{'classifier': 'bert_ktrain', 'test_accuracy': 0.5407, 'precision': 0.5854, 'recall': 0.5407, 'f1_score': 0.5334, 'training_time': 51.72, 'dataset': 'trec_5_subset_train', 'method': None, 'model': None, 'subset_size': 5}
--------------------------------------------------


In [13]:
subsets_df_result.head(20)

,dataset,classifier,model,subset_size,method,test_accuracy,precision,recall,f1_score,training_time
0,atis_5_subset_train,svm_tfidf,None,5,None,0.6055,0.8235,0.6055,0.6602,0.06
1,atis_5_subset_train,svm_glove,None,5,None,0.5815,0.7998,0.5815,0.6436,8.44
2,atis_5_subset_train,lstm_glove,None,5,None,0.3894,0.8163,0.3894,0.4870,37.51
3,atis_5_subset_train,bert_ktrain,None,5,None,0.8388,0.9214,0.8388,0.8625,31.98
4,trec_5_subset_train,svm_tfidf,None,5,None,0.4599,0.6290,0.4599,0.4769,0.28
5,trec_5_subset_train,svm_glove,None,5,None,0.3177,0.4731,0.3177,0.3372,8.95
6,trec_5_subset_train,lstm_glove,None,5,None,0.3803,0.5676,0.3803,0.3964,42.89
7,trec_5_subset_train,bert_ktrain,None,5,None,0.5407,0.5854,0.5407,0.5334,51.72


In [8]:
augmented_atis_df_result = pd.DataFrame(columns=['dataset', 'classifier', 'test_accuracy', 'model', 'subset_size', 'method'])

# Train on subset + matching augmented data
for name in ['ATIS']:
    for subset_size in [5]:
        subset_path = f'NLPProject2024/datasets/{name}/sampled_subsets/ver1/{name.lower()}_{subset_size}_subset.csv'

        for lambada in lambadas:
            for llm in llms:
                if lambada == 'Lambada+Instruct' and llm == 'Mistral':
                    continue
                if lambada == 'Lambada+' and llm == 'GPT2':
                    continue
                if lambada == 'Lambada+Instruct' and llm == 'GPT2':
                    continue

                for classifier in classifiers:
                    directory_path = f'NLPProject2024/filtered_datasets/{lambada}/{llm}/{name}/{classifier}'

                    data_files = glob.glob(os.path.join(directory_path, '*.csv'))

                    for file in data_files:
                      # Hack to train with only matching augmented data
                      if f"{name.lower()}_{subset_size}_" not in file.lower():
                        continue
                      try:
                        dataset_file_name = file.replace(f'{directory_path}', "").replace('.csv', "").replace('/', "")
                        model_path = f'final_results/models/{dataset_file_name}'
                        os.makedirs(model_path, exist_ok=True)
                        print(f"Training {name} with {classifier} on {dataset_file_name}...")
                        X_train, y_train, X_test, y_test, le, X_val, y_val = load_data(subset_path,
                                                                                generated_train_path=file,
                                                                                test_path=f'NLPProject2024/datasets/{name}/{name.lower()}.test.csv',
                                                                                val_path=f'NLPProject2024/datasets/{name}/{name.lower()}.valid.csv')
                        joblib.dump(le, f'{model_path}/label_encoder.pkl')

                        trainer = ModelTrainer(classifier, glove_file, len(le.classes_))
                        train_results = trainer.train_classifier(X_train, y_train, model_path, X_test=X_test, y_test=y_test, X_val=X_val, y_val=y_val)

                        train_results["dataset"] = dataset_file_name
                        train_results["method"] = lambada
                        train_results["model"] = llm
                        train_results["subset_size"] = subset_size
                        augmented_atis_df_result = pd.concat([augmented_atis_df_result, pd.DataFrame([train_results])], ignore_index=True)
                        print(f"Finished training {name} with {classifier} on {dataset_file_name}")
                        print(train_results)
                        print("-" * 50)
                      except Exception as e:
                        print(f"Error training {name} with {classifier} on {dataset_file_name}: {e}")
                        continue

Training ATIS with svm_tfidf on Llama3_8B_ATIS_5_augmented_data_svm_tfidf_filtered...
	Full train path: NLPProject2024/datasets/ATIS/sampled_subsets/ver1/atis_5_subset.csv
	Generated train path: NLPProject2024/filtered_datasets/Lambada/Llama3/ATIS/svm_tfidf/Llama3_8B_ATIS_5_augmented_data_svm_tfidf_filtered.csv
	Test path: NLPProject2024/datasets/ATIS/atis.test.csv
Training svm_tfidf...

Validation Results:
Accuracy: 0.6707
Precision: 0.8934
Recall: 0.6707
F1 Score: 0.7354
Accuracy: 0.6930
Precision: 0.8575
Recall: 0.6930
F1 Score: 0.7358
Model saved.
Finished training ATIS with svm_tfidf on Llama3_8B_ATIS_5_augmented_data_svm_tfidf_filtered
{'classifier': 'svm_tfidf', 'test_accuracy': 0.693, 'precision': 0.8575, 'recall': 0.693, 'f1_score': 0.7358, 'training_time': 0.15, 'dataset': 'Llama3_8B_ATIS_5_augmented_data_svm_tfidf_filtered', 'method': 'Lambada', 'model': 'Llama3', 'subset_size': 5}
--------------------------------------------------
Training ATIS with svm_glove on Llama3_8B_A

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Validation Results:
Accuracy: 0.6415
Precision: 0.8506
Recall: 0.6415
F1 Score: 0.7090
Accuracy: 0.6261
Precision: 0.8045
Recall: 0.6261
F1 Score: 0.6777
Model saved.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Finished training ATIS with svm_glove on Llama3_8B_ATIS_5_augmented_data_svm_glove_filtered
{'classifier': 'svm_glove', 'test_accuracy': 0.6261, 'precision': 0.8045, 'recall': 0.6261, 'f1_score': 0.6777, 'training_time': 8.63, 'dataset': 'Llama3_8B_ATIS_5_augmented_data_svm_glove_filtered', 'method': 'Lambada', 'model': 'Llama3', 'subset_size': 5}
--------------------------------------------------
Training ATIS with lstm_glove on Llama3_8B_ATIS_5_augmented_data_lstm_glove_filtered...
	Full train path: NLPProject2024/datasets/ATIS/sampled_subsets/ver1/atis_5_subset.csv
	Generated train path: NLPProject2024/filtered_datasets/Lambada/Llama3/ATIS/lstm_glove/Llama3_8B_ATIS_5_augmented_data_lstm_glove_filtered.csv
	Test path: NLPProject2024/datasets/ATIS/atis.test.csv
Training lstm_glove...
Epoch 1/20, Loss: 3.0082, Accuracy: 0.0523
Accuracy: 0.0223
Precision: 0.0038
Recall: 0.0223
F1 Score: 0.0065
Validation - Accuracy: 0.0223, Precision: 0.0038, Recall: 0.0223, F1 Score: 0.0065
Epoch 2/20,

...

Model and embeddings saved.
Finished training ATIS with lstm_glove on Llama_8B_lambada_plus_instruct_ATIS_5_augmented_data_lstm_glove_filtered
{'classifier': 'lstm_glove', 'test_accuracy': 0.53, 'precision': 0.7624, 'recall': 0.53, 'f1_score': 0.5986, 'training_time': 37.96, 'dataset': 'Llama_8B_lambada_plus_instruct_ATIS_5_augmented_data_lstm_glove_filtered', 'method': 'Lambada+Instruct', 'model': 'Llama3', 'subset_size': 5}
--------------------------------------------------
Training ATIS with bert_ktrain on Llama_8B_lambada_plus_instruct_ATIS_5_augmented_data_bert_ktrain_filtered...
	Full train path: NLPProject2024/datasets/ATIS/sampled_subsets/ver1/atis_5_subset.csv
	Generated train path: NLPProject2024/filtered_datasets/Lambada+Instruct/Llama3/ATIS/bert_ktrain/Llama_8B_lambada_plus_instruct_ATIS_5_augmented_data_bert_ktrain_filtered.csv
	Test path: NLPProject2024/datasets/ATIS/atis.test.csv
Training bert_ktrain...
preprocessing train...
language: en
train sequence lengths:
	mean : 

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 11
	95percentile : 19
	99percentile : 24




begin training using triangular learning rate policy with max lr of 5e-05...
Epoch 1/20
17/17 [==============================] - 11s 199ms/step - loss: 2.7648 - accuracy: 0.1341 - val_loss: 2.8743 - val_accuracy: 0.1269
Epoch 2/20
17/17 [==============================] - 1s 88ms/step - loss: 1.8228 - accuracy: 0.6951 - val_loss: 2.3432 - val_accuracy: 0.1990
Epoch 3/20
17/17 [==============================] - 1s 91ms/step - loss: 0.8384 - accuracy: 0.8415 - val_loss: 2.3315 - val_accuracy: 0.2024
Epoch 4/20
17/17 [==============================] - 1s 90ms/step - loss: 0.4520 - accuracy: 0.9268 - val_loss: 1.7090 - val_accuracy: 0.5334
Epoch 5/20
17/17 [==============================] - 1s 91ms/step - loss: 0.2576 - accuracy: 0.9756 - val_loss: 1.2370 - val_accuracy: 0.6415
Epoch 6/20
17/17 [==============================] - 1s 88ms/step - loss: 0.1954 - accuracy: 0.9756 - val_loss: 0.9458 - val_accuracy: 0.7736
Epoch 7/20
17/17 [==============================] - 1s 79ms/step - loss: 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.79%
Precision: 0.91
Recall: 0.79
F1 Score: 0.83
Model saved to final_results/models/Llama_8B_lambada_plus_instruct_ATIS_5_augmented_data_bert_ktrain_filtered/bert_ktrain
Finished training ATIS with bert_ktrain on Llama_8B_lambada_plus_instruct_ATIS_5_augmented_data_bert_ktrain_filtered
{'classifier': 'bert_ktrain', 'test_accuracy': 0.7942, 'precision': 0.9064, 'recall': 0.7942, 'f1_score': 0.8329, 'training_time': 38.72, 'dataset': 'Llama_8B_lambada_plus_instruct_ATIS_5_augmented_data_bert_ktrain_filtered', 'method': 'Lambada+Instruct', 'model': 'Llama3', 'subset_size': 5}
--------------------------------------------------


In [9]:
augmented_atis_df_result.sort_values(by=['classifier', 'test_accuracy']).head(50)

,dataset,classifier,test_accuracy,model,subset_size,method,precision,recall,f1_score,training_time
15,Llama3_8B_lambada_plus_ATIS_5_augmented_data_b...,bert_ktrain,0.6913,Llama3,5,Lambada+,0.9220,0.6913,0.7677,35.35
11,GPT2_ATIS_5_augmented_data_bert_ktrain_filtered,bert_ktrain,0.7153,GPT2,5,Lambada,0.8932,0.7153,0.7641,33.99
23,Llama_8B_lambada_plus_instruct_ATIS_5_augmente...,bert_ktrain,0.7942,Llama3,5,Lambada+Instruct,0.9064,0.7942,0.8329,38.72
19,Mistral_7B_lambada_plus_ATIS_5_augmented_data_...,bert_ktrain,0.8233,Mistral,5,Lambada+,0.9310,0.8233,0.8629,31.41
7,Mistral_7B_ATIS_5_augmented_data_bert_ktrain_f...,bert_ktrain,0.8851,Mistral,5,Lambada,0.9243,0.8851,0.8987,31.43
3,Llama3_8B_ATIS_5_augmented_data_bert_ktrain_fi...,bert_ktrain,0.8902,Llama3,5,Lambada,0.9273,0.8902,0.9014,29.58
14,Llama3_8B_lambada_plus_ATIS_5_augmented_data_l...,lstm_glove,0.4528,Llama3,5,Lambada+,0.7974,0.4528,0.5468,37.91
10,GPT2_ATIS_5_augmented_data_lstm_glove_filtered,lstm_glove,0.5163,GPT2,5,Lambada,0.7618,0.5163,0.5935,37.72
18,Mistral_7B_lambada_plus_ATIS_5_augmented_data_...,lstm_glove,0.5214,Mistral,5,Lambada+,0.7456,0.5214,0.5794,37.42
22,Llama_8B_lambada_plus_instruct_ATIS_5_augmente...,lstm_glove,0.5300,Llama3,5,Lambada+Instruct,0.7624,0.5300,0.5986,37.96


In [10]:
augmented_trec_df_result = pd.DataFrame(columns=['dataset', 'classifier', 'test_accuracy', 'model', 'subset_size', 'method'])

# Train on subset + matching augmented data
for name in ['TREC']:
    for subset_size in [5]:
        subset_path = f'NLPProject2024/datasets/{name}/sampled_subsets/ver1/{name.lower()}_{subset_size}_subset.csv'

        for lambada in lambadas:
            for llm in llms:
                if lambada == 'Lambada+Instruct' and llm == 'Mistral':
                    continue
                if lambada == 'Lambada+' and llm == 'GPT2':
                    continue
                if lambada == 'Lambada+Instruct' and llm == 'GPT2':
                    continue

                for classifier in classifiers:
                    directory_path = f'NLPProject2024/filtered_datasets/{lambada}/{llm}/{name}/{classifier}'

                    data_files = glob.glob(os.path.join(directory_path, '*.csv'))

                    for file in data_files:
                      # Hack to train with only matching augmented data
                      if f"{name.lower()}_{subset_size}_" not in file.lower():
                        continue
                      try:
                        dataset_file_name = file.replace(f'{directory_path}', "").replace('.csv', "").replace('/', "")
                        model_path = f'final_results/models/{dataset_file_name}'
                        os.makedirs(model_path, exist_ok=True)
                        print(f"Training {name} with {classifier} on {dataset_file_name}...")
                        X_train, y_train, X_test, y_test, le, X_val, y_val = load_data(subset_path,
                                                                                generated_train_path=file,
                                                                                test_path=f'NLPProject2024/datasets/{name}/{name.lower()}.test.csv',
                                                                                val_path=f'NLPProject2024/datasets/{name}/{name.lower()}.valid.csv')
                        joblib.dump(le, f'{model_path}/label_encoder.pkl')

                        trainer = ModelTrainer(classifier, glove_file, len(le.classes_))
                        train_results = trainer.train_classifier(X_train, y_train, model_path, X_test=X_test, y_test=y_test, X_val=X_val, y_val=y_val)

                        train_results["dataset"] = dataset_file_name
                        train_results["method"] = lambada
                        train_results["model"] = llm
                        train_results["subset_size"] = subset_size
                        augmented_trec_df_result = pd.concat([augmented_trec_df_result, pd.DataFrame([train_results])], ignore_index=True)
                        print(f"Finished training {name} with {classifier} on {dataset_file_name}")
                        print(train_results)
                        print("-" * 50)
                      except Exception as e:
                        print(f"Error training {name} with {classifier} on {dataset_file_name}: {e}")
                        continue

Training TREC with svm_tfidf on Llama3_8B_TREC_5_augmented_data_svm_tfidf_filtered...
	Full train path: NLPProject2024/datasets/TREC/sampled_subsets/ver1/trec_5_subset.csv
	Generated train path: NLPProject2024/filtered_datasets/Lambada/Llama3/TREC/svm_tfidf/Llama3_8B_TREC_5_augmented_data_svm_tfidf_filtered.csv
	Test path: NLPProject2024/datasets/TREC/trec.test.csv
Training svm_tfidf...

Validation Results:
Accuracy: 0.4639
Precision: 0.6540
Recall: 0.4639
F1 Score: 0.4726


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.4956
Precision: 0.7240
Recall: 0.4956
F1 Score: 0.5090
Model saved.
Finished training TREC with svm_tfidf on Llama3_8B_TREC_5_augmented_data_svm_tfidf_filtered
{'classifier': 'svm_tfidf', 'test_accuracy': 0.4956, 'precision': 0.724, 'recall': 0.4956, 'f1_score': 0.509, 'training_time': 1.06, 'dataset': 'Llama3_8B_TREC_5_augmented_data_svm_tfidf_filtered', 'method': 'Lambada', 'model': 'Llama3', 'subset_size': 5}
--------------------------------------------------
Training TREC with svm_glove on Llama3_8B_TREC_5_augmented_data_svm_glove_filtered...
	Full train path: NLPProject2024/datasets/TREC/sampled_subsets/ver1/trec_5_subset.csv
	Generated train path: NLPProject2024/filtered_datasets/Lambada/Llama3/TREC/svm_glove/Llama3_8B_TREC_5_augmented_data_svm_glove_filtered.csv
	Test path: NLPProject2024/datasets/TREC/trec.test.csv
Training svm_glove...

Validation Results:
Accuracy: 0.3273
Precision: 0.4691
Recall: 0.3273
F1 Score: 0.3350


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.3377
Precision: 0.4810
Recall: 0.3377
F1 Score: 0.3504
Model saved.
Finished training TREC with svm_glove on Llama3_8B_TREC_5_augmented_data_svm_glove_filtered
{'classifier': 'svm_glove', 'test_accuracy': 0.3377, 'precision': 0.481, 'recall': 0.3377, 'f1_score': 0.3504, 'training_time': 9.63, 'dataset': 'Llama3_8B_TREC_5_augmented_data_svm_glove_filtered', 'method': 'Lambada', 'model': 'Llama3', 'subset_size': 5}
--------------------------------------------------
Training TREC with lstm_glove on Llama3_8B_TREC_5_augmented_data_lstm_glove_filtered...
	Full train path: NLPProject2024/datasets/TREC/sampled_subsets/ver1/trec_5_subset.csv
	Generated train path: NLPProject2024/filtered_datasets/Lambada/Llama3/TREC/lstm_glove/Llama3_8B_TREC_5_augmented_data_lstm_glove_filtered.csv
	Test path: NLPProject2024/datasets/TREC/trec.test.csv
Training lstm_glove...


...

Accuracy: 0.4505
Precision: 0.6548
Recall: 0.4505
F1 Score: 0.4594
Model and embeddings saved.
Finished training TREC with lstm_glove on Llama_8B_lambada_plus_instruct_TREC_5_augmented_data_lstm_glove_filtered
{'classifier': 'lstm_glove', 'test_accuracy': 0.4505, 'precision': 0.6548, 'recall': 0.4505, 'f1_score': 0.4594, 'training_time': 44.53, 'dataset': 'Llama_8B_lambada_plus_instruct_TREC_5_augmented_data_lstm_glove_filtered', 'method': 'Lambada+Instruct', 'model': 'Llama3', 'subset_size': 5}
--------------------------------------------------
Training TREC with bert_ktrain on Llama_8B_lambada_plus_instruct_TREC_5_augmented_data_bert_ktrain_filtered...
	Full train path: NLPProject2024/datasets/TREC/sampled_subsets/ver1/trec_5_subset.csv
	Generated train path: NLPProject2024/filtered_datasets/Lambada+Instruct/Llama3/TREC/bert_ktrain/Llama_8B_lambada_plus_instruct_TREC_5_augmented_data_bert_ktrain_filtered.csv
	Test path: NLPProject2024/datasets/TREC/trec.test.csv
Training bert_ktrain.

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 10
	95percentile : 17
	99percentile : 21




begin training using triangular learning rate policy with max lr of 5e-05...
Epoch 1/20
65/65 [==============================] - 14s 96ms/step - loss: 3.1224 - accuracy: 0.3606 - val_loss: 3.2001 - val_accuracy: 0.3041
Epoch 2/20
65/65 [==============================] - 4s 67ms/step - loss: 1.2171 - accuracy: 0.8536 - val_loss: 2.7835 - val_accuracy: 0.3799
Epoch 3/20
65/65 [==============================] - 5s 71ms/step - loss: 0.5370 - accuracy: 0.9214 - val_loss: 2.4892 - val_accuracy: 0.4038
Epoch 4/20
65/65 [==============================] - 4s 67ms/step - loss: 0.2819 - accuracy: 0.9584 - val_loss: 2.2357 - val_accuracy: 0.4803
Epoch 5/20
65/65 [==============================] - 4s 66ms/step - loss: 0.1552 - accuracy: 0.9861 - val_loss: 2.2869 - val_accuracy: 0.4815
Epoch 6/20
65/65 [==============================] - 4s 68ms/step - loss: 0.0941 - accuracy: 0.9954 - val_loss: 2.1876 - val_accuracy: 0.5060
Epoch 7/20
65/65 [==============================] - 4s 65ms/step - loss: 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.50%
Precision: 0.66
Recall: 0.50
F1 Score: 0.49
Model saved to final_results/models/Llama_8B_lambada_plus_instruct_TREC_5_augmented_data_bert_ktrain_filtered/bert_ktrain
Finished training TREC with bert_ktrain on Llama_8B_lambada_plus_instruct_TREC_5_augmented_data_bert_ktrain_filtered
{'classifier': 'bert_ktrain', 'test_accuracy': 0.5019, 'precision': 0.6555, 'recall': 0.5019, 'f1_score': 0.4878, 'training_time': 60.43, 'dataset': 'Llama_8B_lambada_plus_instruct_TREC_5_augmented_data_bert_ktrain_filtered', 'method': 'Lambada+Instruct', 'model': 'Llama3', 'subset_size': 5}
--------------------------------------------------


In [11]:
augmented_trec_df_result.sort_values(by=['classifier', 'test_accuracy']).head(40)

,dataset,classifier,test_accuracy,model,subset_size,method,precision,recall,f1_score,training_time
11,GPT2_TREC_5_augmented_data_bert_ktrain_filtered,bert_ktrain,0.4561,GPT2,5,Lambada,0.5651,0.4561,0.4403,72.65
23,Llama_8B_lambada_plus_instruct_TREC_5_augmente...,bert_ktrain,0.5019,Llama3,5,Lambada+Instruct,0.6555,0.5019,0.4878,60.43
7,Mistral_7B_TREC_5_augmented_data_bert_ktrain_f...,bert_ktrain,0.5645,Mistral,5,Lambada,0.6779,0.5645,0.5524,68.38
3,Llama3_8B_TREC_5_augmented_data_bert_ktrain_fi...,bert_ktrain,0.5670,Llama3,5,Lambada,0.7301,0.5670,0.5666,56.85
15,Llama3_8B_lambada_plus_TREC_5_augmented_data_b...,bert_ktrain,0.5971,Llama3,5,Lambada+,0.7345,0.5971,0.5879,61.29
19,Mistral_7B_lambada_plus_TREC_5_augmented_data_...,bert_ktrain,0.6096,Mistral,5,Lambada+,0.7239,0.6096,0.6000,53.18
10,GPT2_TREC_5_augmented_data_lstm_glove_filtered,lstm_glove,0.4110,GPT2,5,Lambada,0.5350,0.4110,0.4217,43.83
22,Llama_8B_lambada_plus_instruct_TREC_5_augmente...,lstm_glove,0.4505,Llama3,5,Lambada+Instruct,0.6548,0.4505,0.4594,44.53
6,Mistral_7B_TREC_5_augmented_data_lstm_glove_fi...,lstm_glove,0.4511,Mistral,5,Lambada,0.5539,0.4511,0.4623,44.87
14,Llama3_8B_lambada_plus_TREC_5_augmented_data_l...,lstm_glove,0.4593,Llama3,5,Lambada+,0.6618,0.4593,0.4704,45.06


In [16]:
df_result = pd.concat([full_df_result, subsets_df_result, augmented_atis_df_result, augmented_trec_df_result], ignore_index=True)
df_result.to_csv("final_results/test_results.csv")

In [18]:
df_result.head(50)

,dataset,model,test_accuracy,precision,recall,f1_score,classifier,training_time,subset_size,method
0,atis_full,NaN,0.9588,0.9516,0.9588,0.9532,svm_tfidf,6.87,NaN,NaN
1,atis_full,NaN,0.9280,0.9121,0.9280,0.9170,svm_glove,11.51,NaN,NaN
2,atis_full,NaN,0.9811,0.9811,0.9811,0.9805,lstm_glove,60.87,NaN,NaN
3,atis_full,NaN,0.9897,0.9888,0.9897,0.9891,bert_ktrain,262.77,NaN,NaN
4,trec_full,NaN,0.9599,0.9611,0.9599,0.9598,svm_tfidf,81.70,NaN,NaN
5,trec_full,NaN,0.8177,0.8224,0.8177,0.8168,svm_glove,40.77,NaN,NaN
6,trec_full,NaN,0.9630,0.9649,0.9630,0.9631,lstm_glove,109.84,NaN,NaN
7,trec_full,NaN,0.9831,0.9838,0.9831,0.9831,bert_ktrain,875.90,NaN,NaN
8,atis_5_subset_train,None,0.6055,0.8235,0.6055,0.6602,svm_tfidf,0.06,5,None
9,atis_5_subset_train,None,0.5815,0.7998,0.5815,0.6436,svm_glove,8.44,5,None
